# Extract and align data from Onix, Harp, Sleap, and photometry

### Setup

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import pandas as pd
import harp

from harp_resources import process, utils
# from sleap import load_and_process as lp #f-string issue not addressed no

### Define paths
Define a root directory (e.g. an experimental session) through wich you can loop to get both photometry data and onix data. 
Check if path names make sense.

In [ ]:
rootdir = '/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1'

#initialize sets to ensure uniqueness
data_paths_set = set()
photometry_paths_set = set()

data_paths = []
photometry_paths = []

for dirpath, subdirs, files in os.walk(rootdir):
    #data paths
    if 'ExperimentEvents' in dirpath:
        trimmed_path = os.path.dirname(dirpath)
        if trimmed_path not in data_paths_set:
            data_paths_set.add(trimmed_path)
            data_paths.append(trimmed_path)
        photometry_path = os.path.join(trimmed_path, "photometry")  # Add "photometry" subfolder
        if photometry_path not in photometry_paths_set:
            photometry_paths_set.add(photometry_path)
            photometry_paths.append(photometry_path)

# print paths sets in notebook for sanity check
print("Data Paths:\n" + "\n".join(data_paths))
print("\nPhotometry Paths:\n" + "\n".join(photometry_paths))

# Call funcitons to extract data and align timestamps
All the function calls needed can be arranged in a function or called one by one.

In [ ]:
def run_extraction_makeh5(data_path, photometry_path, make_h5=False, eyes=False):
    print(f'\n Running extraction for {data_path.split("/")[-1]} from session {data_path.split("/")[-2]}')

    # Load session settings and conversions
    SessionSettings = utils.read_SessionSettings(Path(data_path), print_contents=True)
    conversions = process.calculate_conversions_second_approach(Path(data_path), Path(photometry_path), verbose=False)
    streams = utils.load_registers(Path(data_path))

    # Load photometry data and align timestamps
    Photometry = utils.read_fluorescence(Path(photometry_path))
    Photometry['HARP Timestamps'] = conversions['photometry_to_harp_time'](Photometry.index)

    # Load OnixAnalog data
    OnixAnalogClock = utils.read_OnixAnalogClock(Path(data_path))
    OnixAnalogData = utils.read_OnixAnalogData(Path(data_path), binarise=True)
    ExperimentEvents = utils.read_ExperimentEvents(Path(data_path))
    photodiode_series = pd.Series(OnixAnalogData[:, 0], index=conversions['onix_to_harp_timestamp'](OnixAnalogClock))

    print('Adding Photometry, Eye Movements, and Photodiode to the streams')
    # Add Photometry and Photodiode streams
    streams = process.reformat_and_add_many_streams(
        streams, Photometry, 'Photometry', ['470_dfF', 'z_470'], index_column_name='HARP Timestamps'
    )
    streams = process.add_stream(streams, 'ONIX', photodiode_series, 'Photodiode')

    # Process videography data if eyes=True
    if eyes:
        print('  Checking for and processing videography data...')
        try:
            # Load videography data
            VideoData1, VideoData2, VideoData1_Has_Sleap, VideoData2_Has_Sleap = lp.load_videography_data(data_path)

            if VideoData2_Has_Sleap:
                print('  Processing VideoData2 with SLEAP data...')

                # Interpolate missing data
                VideoData2 = VideoData2.interpolate()

                # Extract coordinates and compute transformations
                columns_of_interest = [
                    'left.x', 'left.y', 'center.x', 'center.y', 'right.x', 'right.y',
                    'p1.x', 'p1.y', 'p2.x', 'p2.y', 'p3.x', 'p3.y', 'p4.x', 'p4.y',
                    'p5.x', 'p5.y', 'p6.x', 'p6.y', 'p7.x', 'p7.y', 'p8.x', 'p8.y'
                ]
                coordinates_dict = lp.get_coordinates_dict(VideoData2, columns_of_interest)

                # Calculate transformations
                theta = lp.find_horizontal_axis_angle(VideoData2, 'left', 'center')
                center_point = lp.get_left_right_center_point(coordinates_dict)

                reformatted_coordinates_dict = lp.get_reformatted_coordinates_dict(coordinates_dict, ['left', 'right', 'center'] + [f'p{i}' for i in range(1, 9)])
                centered_coordinates_dict = lp.get_centered_coordinates_dict(reformatted_coordinates_dict, center_point)
                rotated_coordinates_dict = lp.get_rotated_coordinates_dict(centered_coordinates_dict, theta)

                # Extract ellipse parameters
                columns_of_interest = [f'p{i}' for i in range(1, 9)]
                ellipse_parameters_data, ellipse_center_points_data = lp.get_fitted_ellipse_parameters(
                    rotated_coordinates_dict, columns_of_interest
                )

                # Compute additional metrics
                average_diameter = np.mean([ellipse_parameters_data[:, 0], ellipse_parameters_data[:, 1]], axis=0)

                # Prepare SLEAP video data for streams
                SleapVideoData2 = process.convert_arrays_to_dataframe(
                    ['Seconds', 'Ellipse.Diameter', 'Ellipse.Angle', 'Ellipse.Center.X', 'Ellipse.Center.Y'],
                    [VideoData2['Seconds'].values, average_diameter, ellipse_parameters_data[:, 2],
                     ellipse_center_points_data[:, 0], ellipse_center_points_data[:, 1]]
                )

                streams = process.reformat_and_add_many_streams(
                    streams, SleapVideoData2, 'SleapVideoData2',
                    ['Ellipse.Diameter', 'Ellipse.Angle', 'Ellipse.Center.X', 'Ellipse.Center.Y']
                )

        except Exception as e:
            print(f"Error processing videography data: {e}")

    # Display timepoint info for streams
    _ = process.get_timepoint_info(streams, print_all=True)

    # Resample streams
    resampled_streams = process.pad_and_resample(streams, resampling_period='1 ms', method='linear')
    _ = process.get_timepoint_info(resampled_streams, print_all=True)

    # Apply unit conversions for optical tracking sensor streams
    print('  Applying linear and angular conversion to Optical tracking sensor streams (cm/sec and degrees/sec)')
    resampled_streams['H1']['OpticalTrackingRead0X(46)'] = process.running_unit_conversion(
        resampled_streams['H1']['OpticalTrackingRead0X(46)'] * 100
    )
    resampled_streams['H1']['OpticalTrackingRead0Y(46)'] = process.rotation_unit_conversion(
        resampled_streams['H1']['OpticalTrackingRead0Y(46)']
    )

    print(' - Streams are extracted and can be used or made to h5')

    if make_h5:
        # Define streams to save, including SLEAP data if processed
        streams_to_save_pattern = {
            'Photometry': ['470_dfF', 'z_470'],
            'ONIX': ['Photodiode'],
        }
        if eyes:
            streams_to_save_pattern['SleapVideoData2'] = ['Ellipse.Diameter', 'Ellipse.Center.X', 'Ellipse.Center.Y']

        process.save_streams_as_h5(Path(data_path), resampled_streams, streams_to_save_pattern, SessionSettings)
        print('Streams saved as h5 file \n')

    return data_path, resampled_streams, streams_to_save_pattern



In [ ]:
SessionSettings = utils.read_SessionSettings(Path(data_paths[0]), print_contents=True)

In [ ]:
SessionSettings

In [ ]:
session_settings_reader = utils.SessionData("SessionSettings")
session_settings = utils.load_vers2_3(session_settings_reader, data_paths[0])

In [ ]:
import json
json.dumps(session_settings, indent=4)

## Call the function
Below, the funciton is called with only one path combo, and then with several through a loop
If all seems to work, you can set make_h5 to True, or you can use the resampled streams and save pattern daved to the dict made in the loop to make the h5 files.

In [ ]:
data_path, resampled_streams, streams_to_save_pattern = run_extraction_makeh5(
    data_paths[1], photometry_paths[1], make_h5=False, eyes=True
)

In [ ]:
stream_dict = {}
for i, (datapath, photometry_path) in enumerate(zip(data_paths, photometry_paths)):
    print(datapath)
    print(photometry_path)
    stream_dict[f'dataset_{i}'] = {}
    data_path, resampled_streams, streams_to_save_pattern = run_extraction_makeh5(
    data_paths[1], photometry_paths[1], make_h5=False, eyes=True)
    stream_dict[f'dataset_{i}']['resampled_streams'] = resampled_streams
    stream_dict[f'dataset_{i}']['streams_to_save_pattern'] = streams_to_save_pattern
    stream_dict[f'dataset_{i}']['data_path'] = data_path
    

In [ ]:
#!!! ONLY RUN IF YOU WANT TO MAKE NEW H5 FILES !!!!

for dataset, data_dict in stream_dict.items():

    process.save_streams_as_h5(Path(data_dict['data_path']), data_dict['resampled_streams'], data_dict['streams_to_save_pattern'], SessionSettings)
    

# Extracting Noras data

This is adapted from the instructions given by Andrew for reading the temporary onix digital file versions (the weird ones)

In [ ]:
photometry_path = '/Volumes/RanczLab/Nora_Cohort1_training/Training_day4/B6J2717-2024-11-28T09-37-55/photometry'
data_path = '/Volumes/RanczLab/Nora_Cohort1_training/Training_day4/B6J2717-2024-11-28T09-37-55/'

photometry_path = '/Volumes/RanczLab/Nora_Cohort1_training/Visual_mismatch_day1/B6J2717/photometry_processed'
data_path = '/Volumes/RanczLab/Nora_Cohort1_training/Visual_mismatch_day1/B6J2717/'

h1_datafolder = data_path+'HarpDataH1'
h2_datafolder = data_path+'HarpDataH2'

In [ ]:
pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv') #Processed_fluorescence.csv #Fluorescence.csv

In [ ]:
# from andrew:
#Changed didgital output file
h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)
session_data_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader = utils.Video("VideoData1")
onix_digital_reader = utils.TimestampedCsvReader("OnixDigital", columns=["Clock", "HubClock", 
                                                                         "DigitalInputs0",
                                                                         "DigitalInputs1",
                                                                         "DigitalInputs2",
                                                                         "DigitalInputs3",
                                                                         "DigitalInputs4",
                                                                         "DigitalInputs5"
                                                                         "DigitalInputs6",
                                                                         "DigitalInputs7",
                                                                         "DigitalInputs8",
                                                                         "Buttons"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

In [ ]:
# read metadata
session_settings = utils.load_2(session_data_reader, data_path)

print(session_settings.iloc[0]['metadata'].blocks[0].haltProtocol)